Load libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
import pandas as pd
import os

df_metadata = pd.read_csv('metadata_licenses.csv', sep=',')

In [ ]:
len(df_metadata)

11774

Function definition for extracting Author, Publishing Date, Modified Date, and DO

In [ ]:
def extract_author_date_doi_pub(url):
    # Send a GET request to the URL and retrieve the HTML content
    response = requests.get(url)
    html_content = response.text
    #print(html_content)
    date_mod=''
    author=''
    doi=''
    pub_date=''
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the license element using its HTML tag or attribute
    #license_element = soup.find('License')  # Replace 'license' with the appropriate tag or attribute

    date = soup.find('meta', attrs={'name': 'dc.datemodified'})

    if date is not None:
        date_mod = date['content'].strip()

    if not date_mod:
      date_mod = 'No info'

    contr = soup.find('meta', attrs={'name': 'dc.creator'})
    if contr is not None:
        author = contr['content'].strip()

    if not author:
      author = 'No info'

    citation = soup.find('meta', attrs={'name': 'citation_doi'})
    if citation is not None:
        doi = citation['content'].strip()

    if not doi:
      doi = 'No info'

    citation_pub_date = soup.find('meta', attrs={'name': 'citation_publication_date'})
    if citation_pub_date is not None:
        pub_date = citation_pub_date['content'].strip()

    if not pub_date:
      pub_date = 'No info'

    return author, date_mod, doi, pub_date

General Function for extracting Meta-Data

In [ ]:
os.listdir(path)

In [ ]:
def extract_general_metadata_from_r_code(df, path):
        # Listing all files in the directory

    for filename in tqdm(os.listdir(path)):
      file_path = os.path.join(path, filename)
      if os.path.isfile(file_path):
        try:
          r_code = open(file_path, 'r', encoding='utf-8').read()
        except:
          r_code = open(file_path, 'r', encoding='latin-1').read()
        #r_code = open(file_path).read()
        index = int(filename.split('_')[0])

        # Author, Date modified and DOI
        Author, Date_modified, DOI, Publication_date = extract_author_date_doi_pub(df_metadata['Link'][index])
        # Source
        Source = df_metadata['Link'][index]
        # Title
        Title = df_metadata['Title'][index]
        # Domain
        Domain = df_metadata['Category'][index]
        # License
        License = df_metadata['License'][index]

        # Extracting package names
        packages = re.findall(r'library\((\w+)\)', r_code) + re.findall(r'require\((\w+)\)', r_code)

        if packages:
            Packages_ = packages
        else:
          Packages_ = 'None'

        # Extracting dataset names from various read functions
        Datasets_ = list(set(
            re.findall(r'read\.csv\(file="([^"]+)"', r_code) +  # For read.csv, read.csv2, read.table, etc.
            re.findall(r'read.\w+\("([^"]+)"', r_code)+
            re.findall(r'read_\w+\(["\']([^"\']+)["\']', r_code) +  #  From the 'readxl' package
            re.findall(r'load\(["\']([^"\']+)["\']', r_code)+  # For load function
            re.findall(r'readRDS\(["\']([^"\']+)["\']', r_code)+ # Reads an RDS file
            re.findall(r'fread\(["\']([^"\']+)["\']', r_code)+ # From the 'data.table' package
            re.findall(r'readLines\(["\']([^"\']+)["\']', r_code)
        ))

        if Datasets_:
            Datasets_ = Datasets_
        else:
          Datasets_ = 'None'

        # Extracting output types from various write functions
        output_files = re.findall(r'write\.\w+\(\w+, "([^"]+)"', r_code)
        output_files += re.findall(r'save\.image\(file="([^"]+)"\)', r_code)
        output_files += re.findall(r'save\.image\("([^"]+)"', r_code)
        output_files += re.findall(r'saveRDS\("([^"]+)"', r_code)
        output_files += re.findall(r'save\("([^"]+)"', r_code)
        output_files += re.findall(r'ggsave\("([^"]+)"', r_code)
        output_files += re.findall(r'sink\("([^"]+)"', r_code)
        output_files += re.findall(r'dput\("([^"]+)"', r_code)
        output_files += re.findall(r'write\.table\([^,]+,\s*file="([^"]+)"', r_code)
        output_files += re.findall(r'paste\(\w+, "([^"]+)", sep="/"\)', r_code)

        Output_Types = ""
        if output_files:
          try:

            Output_Types = list(set(re.search(r'\.(\w+)$', file).group(1).upper() for file in output_files))
          except:
            continue
        Output_Names = set([output for output in output_files])


        df.loc[len(df.index)] = [Author , Datasets_ , Packages_ , set(Output_Types) , set(Output_Names) ,Source , Title, Domain , License ,Publication_date , Date_modified, DOI, filename]
        Author, Datasets_, Packages_, Output_Types, Output_Names, Source, Title, Domain, License, Publication_date, Date_modified, DOI, filename = "", "", "", "", "", "", "", "", "", "", "", "", ""
    return df

Inference

In [ ]:
# Extract and Print metadata
#pd.DataFrame([extract_general_metadata_from_r_code('/content/drive/MyDrive/CodeInspector/Data/OSF data/OSF R files/0 Illustration CRTs.R')])*
df = pd.DataFrame({"Author": [],  "Datasets": [], "Packages": [], "Output Types": [], "Output Names": [], "Source": [],   "Title": [],  "Domain": [], "License": [], "Publication date": [], "Date modified": [],"DOI": [], "File name": []})
Files_metadata_table = extract_general_metadata_from_r_code (df, "/content/drive/MyDrive/CodeInspector/Data/Final_versions_of_OSF_data/OSF_R_Code")

100%|██████████| 7566/7566 [3:21:02<00:00,  1.59s/it]


In [ ]:
Files_metadata_table.to_csv('OSF_Meta_Data_v0.1.csv', index=False)

In [ ]:
Files_metadata_table.head(10)

In [ ]:
len(Files_metadata_table)

7555

In [ ]:
"""
    metadata = {
        "Author": [],  # Typically not available in code comments
        "Datasets": [],
        "Packages": [],
        "Output Types": [],
        "Output Names": [],
        "Source": [],  # not available in code comments
        "Title": [], # not available in code comments
        "Domain": [], # not available in code comments
        "License": [], # not available in code comments
        "Publication date":[], # not available in code comments
        "Date modified": [], # not available in code comments
        "DOI": [], # not available in code comments
        "File name": []

    }

"""

# Get just the description and wiki data

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
import pandas as pd
import os

df_metadata = pd.read_csv('/content/StatCodeSeach-Code-Comment-Full-Meta-Data.csv', sep=',')

In [ ]:
df_metadata['Project description'] = ''

In [ ]:
def extract_desc_wiki(url):
    # Send a GET request to the URL and retrieve the HTML content
    response = requests.get(url)
    html_content = response.text
    #print(html_content)
    desc_mod=''
    wiki=''

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the license element using its HTML tag or attribute
    #license_element = soup.find('License')  # Replace 'license' with the appropriate tag or attribute

    desc = soup.find('meta', attrs={'name': 'dc.abstract'})
    #print(desc)
    if desc is not None:
        desc_mod = desc['content'].strip()

    if not desc_mod:
      desc_mod = 'No info'

    return desc_mod

for i in range(len(df_metadata)):
  print(i)
  df_metadata['Project description'][i] = extract_desc_wiki(df_metadata['Source'][i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
df_metadata.to_csv('StatCodeSearch-Code-Comment-Metadata-V0.csv', index=False)

In [ ]:
df_metadata.head(10)